In [9]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from matplotlib.colors import Normalize
import re
from scipy.spatial import cKDTree
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
from matplotlib.ticker import ScalarFormatter




In [13]:
def smooth_data(data, sigma=2):
    return gaussian_filter1d(data, sigma)

def create_roi_mask(image_shape, center, axes):
    y, x = np.ogrid[:image_shape[0], :image_shape[1]]
    mask = ((x - center[0]) / axes[0]) ** 2 + ((y - center[1]) / axes[1]) ** 2 <= 1
    return mask

def find_oval_points(center, boundary_points, num_points, oval_width, oval_height):
    angles = np.linspace(0, 2*np.pi, num_points, endpoint=False)
    
    ellipse_points = np.column_stack((
        center[0] + (oval_width / 2) * np.cos(angles),
        center[1] + (oval_height / 2) * np.sin(angles)
    ))
    
    tree = cKDTree(boundary_points)
    _, indices = tree.query(ellipse_points)
    
    oval_points = boundary_points[indices]
    
    return oval_points

def intensity_to_density(intensity):
    return (intensity + 39.76) / 143.05

def calculate_edge_flux_and_plot(tiff_folder_path, df, pixel_size, incubation_time, replicate, num_points=64, oval_width=1600, oval_height=600, center_offset=(0, 0), gridSize=30):
    df_filtered = df[(df['incubation_time'] == incubation_time) & (df['replicate'] == replicate)]
    
    if df_filtered.empty:
        raise ValueError(f"No data found for incubation time {incubation_time} and replicate {replicate}")
    
    tiff_files = sorted([f for f in os.listdir(tiff_folder_path) if 'Cy5_' in f and f.endswith('.tif')])
    
    # Read the first image to get dimensions
    first_image_path = os.path.join(tiff_folder_path, tiff_files[0])
    first_image = Image.open(first_image_path)
    first_image_np = np.array(first_image)
    height, width = first_image_np.shape
    
    # Calculate ROI center and create mask
    roi_center = (width // 2 + center_offset[0], height // 2 + center_offset[1])
    roi_axes = (oval_width // 2, oval_height // 2)
    roi_mask = create_roi_mask((height, width), roi_center, roi_axes)
    
    y, x = np.where(roi_mask)
    boundary_points = np.column_stack((x, y))
    
    key_points = find_oval_points(roi_center, boundary_points, num_points, oval_width, oval_height)
    
    flux_data = np.zeros((num_points, len(tiff_files)))
    
    for frame, tiff_file in enumerate(tiff_files):
        if frame >= len(df_filtered):
            print(f"No data found for frame {frame}")
            continue
        
        img = np.array(Image.open(os.path.join(tiff_folder_path, tiff_file)))
        
        frame_data = df_filtered.iloc[frame]
        ux, uy = frame_data['ux'], frame_data['uy']
        
        point_indices = (key_points[:, 1] // gridSize, key_points[:, 0] // gridSize)
        vx = ux[point_indices]
        vy = uy[point_indices]
        
        intensities = img[key_points[:, 1], key_points[:, 0]]
        densities = intensity_to_density(intensities)
        
        normals = key_points - roi_center
        normals = normals / np.linalg.norm(normals, axis=1)[:, np.newaxis]
        flux = np.where(key_points[:, 0] < roi_center[0],
                        vx * densities * pixel_size,
                        -vx * densities * pixel_size)
        
        flux_data[:, frame] = flux
    
    flux_data = smooth_data(flux_data, sigma=2)
    
    cumulative_flux = np.sum(flux_data, axis=1)
    
    total_flux = np.sum(flux_data, axis=0)
    
    return key_points, flux_data, cumulative_flux, total_flux, tiff_files[-1], roi_center, roi_mask

def process_all_videos(root_folder, df, pixel_size, num_points, oval_width, oval_height, center_offset):
    results = {}
    
    for folder in os.listdir(root_folder):
        match = re.search(r'correlation_(\d+)_min', folder)
        if match:
            incubation_time = int(match.group(1))
            folder_path = os.path.join(root_folder, folder)
            
            for pos_folder in os.listdir(folder_path):
                if pos_folder.startswith('Pos'):
                    replicate = int(pos_folder[3:]) + 1
                    tiff_folder_path = os.path.join(folder_path, pos_folder)
                    
                    try:
                        key_points, flux_data, cumulative_flux, total_flux, last_tiff, roi_center, roi_mask = calculate_edge_flux_and_plot(
                            tiff_folder_path, df, pixel_size, incubation_time, replicate,
                            num_points=num_points, oval_width=oval_width, oval_height=oval_height, center_offset=center_offset
                        )
                        results[(incubation_time, replicate)] = (key_points, flux_data, cumulative_flux, total_flux, last_tiff, tiff_folder_path, roi_center, roi_mask)
                    except Exception as e:
                        print(f"Error processing {tiff_folder_path}: {str(e)}")
    
    return results

In [14]:
def extract_info_from_folder(folder_name):
    # Extract incubation time and replicate number
    match = re.match(r'(\d+)(?:_min)?_(\d+)_finer', folder_name)
    if match:
        incubation_time = int(match.group(1))
        replicate = int(match.group(2))
        return incubation_time, replicate
    return None, None

def load_piv_data(root_directory):
    data = []
    
    # Iterate through all items in the root directory
    for item in os.listdir(root_directory):
        item_path = os.path.join(root_directory, item)
        if os.path.isdir(item_path) and item.endswith('_finer'):
            # Extract incubation time and replicate number
            incubation_time, replicate = extract_info_from_folder(item)
            
            if incubation_time is not None and replicate is not None:
                velocity_file = os.path.join(item_path, 'velocity_data.pkl')
                
                if os.path.exists(velocity_file):
                    # Load velocity data
                    with open(velocity_file, 'rb') as f:
                        velocity_data = pickle.load(f)
                    
                    # Process velocity data
                    for frame, (ux, uy) in velocity_data.items():
                        mean_velocity = np.sqrt(np.mean(ux**2 + uy**2))
                        max_velocity = np.sqrt(np.max(ux**2 + uy**2))
                        
                        data.append({
                            'folder_name': item,
                            'incubation_time': incubation_time,
                            'replicate': replicate,
                            'frame': frame,
                            'mean_velocity': mean_velocity,
                            'max_velocity': max_velocity,
                            'ux': ux,
                            'uy': uy
                        })
    
    # Create DataFrame
    df = pd.DataFrame(data)
    return df

# Usage
root_directory = "/Users/scliu/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/0104PIV_data_fine"
df = load_piv_data(root_directory)


# Display the first few rows of the DataFrame
print(df.head())

# Get basic statistics
print(df.describe())

# Group by incubation time and replicate, and get mean velocities
summary = df.groupby(['incubation_time', 'replicate']).agg({
    'mean_velocity': 'mean',
    'max_velocity': 'mean'
}).reset_index()

print(summary)

     folder_name  incubation_time  replicate  frame  mean_velocity  \
0  5_min_1_finer                5          1      0       2.138180   
1  5_min_1_finer                5          1      1       2.119352   
2  5_min_1_finer                5          1      2       2.231920   
3  5_min_1_finer                5          1      3       2.279929   
4  5_min_1_finer                5          1      4       2.213320   

   max_velocity                                                 ux  \
0      7.415277  [[2.2125442, 2.9422464, 2.7424815, 1.3800211, ...   
1      7.632476  [[-0.04790093, 0.10697776, -0.42355973, -1.231...   
2     10.642627  [[0.8120931, 0.29884127, -0.8528447, -1.763579...   
3      8.440644  [[0.9665268, -0.38477817, -2.4426982, -3.44039...   
4      8.004435  [[-0.15921363, -0.040638622, 0.265877, 0.38512...   

                                                  uy  
0  [[0.64672816, 0.2536938, -0.5066219, -1.139675...  
1  [[0.096666045, -0.18312608, -0.24286759, -0.0

In [15]:
# Usage
root_folder = "/Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/0104"
pixel_size = 0.43  # µm per pixel
num_points = 256
oval_width = 1400  # pixels
oval_height = 500  # pixels
center_offset = (160, 0)  # Adjust these values to move the ROI

# Process all videos
results = process_all_videos(root_folder, df, pixel_size, num_points, oval_width, oval_height, center_offset)

# Find global min and max for colorbar
global_min = min(np.min(result[2]) for result in results.values())
global_max = max(np.max(result[2]) for result in results.values())

No data found for frame 118
No data found for frame 119
No data found for frame 118
No data found for frame 119
No data found for frame 118
No data found for frame 119
No data found for frame 118
No data found for frame 119
No data found for frame 118
No data found for frame 119
No data found for frame 118
No data found for frame 119
No data found for frame 118
No data found for frame 119
No data found for frame 118
No data found for frame 119
No data found for frame 118
No data found for frame 119
No data found for frame 118
No data found for frame 119
No data found for frame 118
No data found for frame 119
No data found for frame 118
No data found for frame 119
No data found for frame 118
No data found for frame 119
No data found for frame 118
No data found for frame 119
No data found for frame 118
No data found for frame 119
No data found for frame 118
No data found for frame 119
No data found for frame 118
No data found for frame 119
No data found for frame 118
No data found for fr

In [16]:
# Plotting
for (incubation_time, replicate), (key_points, flux_data, cumulative_flux, total_flux, last_tiff, tiff_folder_path, roi_center, roi_mask) in results.items():
    last_frame = np.array(Image.open(os.path.join(tiff_folder_path, last_tiff)))

    # Apply scaling factor to adjust image brightness
    scaling_factor = 0.3  # Adjust this value to change brightness
    last_frame_scaled = np.clip(last_frame * scaling_factor, 0, 255).astype(np.uint8)

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(30, 10))
    
    # Oval plot
    ax1.imshow(last_frame_scaled, cmap='gray', aspect='equal')
    scatter = ax1.scatter(key_points[:, 0], key_points[:, 1], c=cumulative_flux, cmap='coolwarm', s=15, alpha = 0.5, norm=Normalize(vmin=global_min, vmax=global_max))
    plt.colorbar(scatter, ax=ax1, label='Cumulative Flux (MT/µm)')
    ax1.set_title(f'Cumulative Flux on ROI Oval\nIncubation Time: {incubation_time} min, Replicate: {replicate}')
    ax1.axis('equal')
    ax1.set_xlim(0, last_frame.shape[1])
    ax1.set_ylim(last_frame.shape[0], 0)  # Reverse y-axis to match image coordinates

    # Flux vs time plot for key points
    time_points = np.arange(flux_data.shape[1]) * 10  # Convert frames to seconds
    ax2.plot(time_points, flux_data[0], label='Left')
    ax2.plot(time_points, flux_data[num_points//4], label='Top')
    ax2.plot(time_points, flux_data[num_points//2], label='Right')
    ax2.plot(time_points, flux_data[3*num_points//4], label='Bottom')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Flux (MT/µm/s)')
    ax2.set_title('Flux vs Time for Key Points')
    ax2.legend()

    # Total flux over time plot
    ax3.plot(time_points, total_flux)
    ax3.set_xlabel('Time (s)')
    ax3.set_ylabel('Total Flux (MT/µm/s)')
    ax3.set_title('Total Flux around ROI over Time')
    ax3.axhline(0, color='gray', linewidth=0.5, linestyle='--')

    plt.tight_layout()
    plt.savefig(f'/Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/flux/flux_plot_{incubation_time}min_rep{replicate}.png', dpi=300)
    plt.close()

print("All plots saved.")

All plots saved.


In [46]:
# Create a new figure for the combined total flux plot
fig_combined, ax_combined = plt.subplots(figsize=(20, 12))

# Plotting data for combined plot
for (incubation_time, replicate), (key_points, flux_data, cumulative_flux, total_flux, last_tiff, tiff_folder_path, roi_center, roi_mask) in results.items():
    time_points = np.arange(total_flux.shape[0]) * 10  # Convert frames to seconds
    color = color_map(norm(incubation_time))
    ax_combined.plot(time_points, total_flux, color=color, alpha=0.7)

# Finalize combined plot
ax_combined.set_xlabel('Time (s)', fontsize=18)
ax_combined.set_ylabel('Total Flux (MT/µm/s)', fontsize=18)
ax_combined.set_title('Total Flux around ROI over Time for All Incubation Times', fontsize=20)
ax_combined.axhline(0, color='gray', linewidth=0.5, linestyle='--')
ax_combined.tick_params(axis='both', which='major', labelsize=16)

# Add colorbar for incubation times
sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax_combined)
cbar.set_label('Incubation Time (min)', fontsize=18)
cbar.ax.tick_params(labelsize=16)

plt.tight_layout()
plt.savefig('/Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/flux/combined_total_flux_plot.png', dpi=300, bbox_inches='tight')
plt.close()

print("Combined total flux plot saved with color map only.")

Combined total flux plot saved with color map only.


In [31]:
def set_plot_style():
    plt.style.use('default')
    plt.rc('font', size=24)          # controls default text sizes
    plt.rc('axes', titlesize=24)     # fontsize of the axes title
    plt.rc('axes', labelsize=18)     # fontsize of the x and y labels
    plt.rc('xtick', labelsize=18)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=18)    # fontsize of the tick labels
    plt.rc('legend', fontsize=22)    # legend fontsize
    plt.rc('figure', titlesize=22)   # fontsize of the figure title

    return {
        'FIGURE_WIDTH': 17,
        'FIGURE_HEIGHT': 5,
        'SCATTER_SIZE': 20,
        'SCATTER_ALPHA': 0.7
    }

# Apply the plot style
style_params = set_plot_style()

# Create a new figure for the combined total flux plot
fig_combined, ax_combined = plt.subplots(figsize=(style_params['FIGURE_WIDTH'], style_params['FIGURE_HEIGHT']))

# Use the 'coolwarm' colormap
color_map = plt.cm.coolwarm

# Plotting data for combined plot
for (incubation_time, replicate), (key_points, flux_data, cumulative_flux, total_flux, last_tiff, tiff_folder_path, roi_center, roi_mask) in results.items():
    time_points = np.arange(total_flux.shape[0]) * 10  # Convert frames to seconds
    color = color_map(norm(incubation_time))
    ax_combined.plot(time_points, total_flux, color=color, alpha=style_params['SCATTER_ALPHA'])

# Finalize combined plot
ax_combined.set_xlabel('Time (s)')
ax_combined.set_ylabel('Total Flux (MT/s)')
ax_combined.set_title('Total Flux around ROI over Time for All Incubation Times')
ax_combined.axhline(0, color='gray', linewidth=0.5, linestyle='--')

# Format y-axis to use scientific notation
ax_combined.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax_combined.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

# Add colorbar for incubation times
sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax_combined)
cbar.set_label('Incubation Time (min)')

plt.tight_layout()
plt.savefig('/Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/flux/combined_total_flux_plot.png', dpi=300, bbox_inches='tight')
plt.close()

print("Combined total flux plot saved with coolwarm color map.")



Combined total flux plot saved with coolwarm color map.


## Mass flux

In [50]:
import numpy as np
from scipy.spatial import cKDTree
import os
import pandas as pd
import re
import pickle
from scipy.ndimage import gaussian_filter1d
from PIL import Image

def smooth_data(data, sigma=2):
    return gaussian_filter1d(data, sigma)

def create_roi_mask(image_shape, center, axes):
    y, x = np.ogrid[:image_shape[0], :image_shape[1]]
    mask = ((x - center[0]) / axes[0]) ** 2 + ((y - center[1]) / axes[1]) ** 2 <= 1
    return mask

def find_oval_points(center, boundary_points, num_points, oval_width, oval_height, grid_size):
    angles = np.linspace(0, 2*np.pi, num_points, endpoint=False)
    
    ellipse_points = np.column_stack((
        center[0] + (oval_width / 2) * np.cos(angles),
        center[1] + (oval_height / 2) * np.sin(angles)
    ))
    
    tree = cKDTree(boundary_points)
    _, indices = tree.query(ellipse_points)
    
    # Ensure only one point per grid
    unique_indices = []
    seen_grids = set()
    for idx in indices:
        grid_x, grid_y = boundary_points[idx] // grid_size
        if (grid_x, grid_y) not in seen_grids:
            seen_grids.add((grid_x, grid_y))
            unique_indices.append(idx)
    
    oval_points = boundary_points[unique_indices]
    
    return oval_points

def intensity_to_density(intensity):
    return (intensity + 39.76) / 143.05

def calculate_edge_flux_and_plot(tiff_folder_path, df, pixel_size, incubation_time, replicate, num_points=64, oval_width=1600, oval_height=600, center_offset=(0, 0), grid_size=30):
    df_filtered = df[(df['incubation_time'] == incubation_time) & (df['replicate'] == replicate)]
    
    if df_filtered.empty:
        raise ValueError(f"No data found for incubation time {incubation_time} and replicate {replicate}")
    
    tiff_files = sorted([f for f in os.listdir(tiff_folder_path) if 'Cy5_' in f and f.endswith('.tif')])
    
    # Read the first image to get dimensions
    first_image_path = os.path.join(tiff_folder_path, tiff_files[0])
    first_image = Image.open(first_image_path)
    first_image_np = np.array(first_image)
    height, width = first_image_np.shape
    
    # Calculate ROI center and create mask
    roi_center = (width // 2 + center_offset[0], height // 2 + center_offset[1])
    roi_axes = (oval_width // 2, oval_height // 2)
    roi_mask = create_roi_mask((height, width), roi_center, roi_axes)
    
    y, x = np.where(roi_mask)
    boundary_points = np.column_stack((x, y))
    
    key_points = find_oval_points(roi_center, boundary_points, num_points, oval_width, oval_height, grid_size)
    
    flux_data = np.zeros((len(key_points), len(tiff_files)))
    
    for frame, tiff_file in enumerate(tiff_files):
        if frame >= len(df_filtered):
            print(f"No data found for frame {frame}")
            continue
        
        img = np.array(Image.open(os.path.join(tiff_folder_path, tiff_file)))
        
        frame_data = df_filtered.iloc[frame]
        ux, uy = frame_data['ux'], frame_data['uy']
        
        point_indices = (key_points[:, 1] // grid_size, key_points[:, 0] // grid_size)
        vx = ux[point_indices]
        vy = uy[point_indices]
        
        intensities = img[key_points[:, 1], key_points[:, 0]]
        densities = intensity_to_density(intensities)
        
        normals = key_points - roi_center
        normals = normals / np.linalg.norm(normals, axis=1)[:, np.newaxis]
        flux = np.where(key_points[:, 0] < roi_center[0],
                        vx * densities * pixel_size,
                        -vx * densities * pixel_size)
        
        flux_data[:, frame] = flux
    
    flux_data = smooth_data(flux_data, sigma=2)
    
    cumulative_flux = np.sum(flux_data, axis=1)
    
    total_flux = np.sum(flux_data, axis=0)
    
    return key_points, flux_data, cumulative_flux, total_flux, tiff_files[-1], roi_center, roi_mask

def process_all_videos(root_folder, df, pixel_size, num_points, oval_width, oval_height, center_offset):
    results = {}
    
    for folder in os.listdir(root_folder):
        match = re.search(r'correlation_(\d+)_min', folder)
        if match:
            incubation_time = int(match.group(1))
            folder_path = os.path.join(root_folder, folder)
            
            for pos_folder in os.listdir(folder_path):
                if pos_folder.startswith('Pos'):
                    replicate = int(pos_folder[3:]) + 1
                    tiff_folder_path = os.path.join(folder_path, pos_folder)
                    
                    try:
                        key_points, flux_data, cumulative_flux, total_flux, last_tiff, roi_center, roi_mask = calculate_edge_flux_and_plot(
                            tiff_folder_path, df, pixel_size, incubation_time, replicate,
                            num_points=num_points, oval_width=oval_width, oval_height=oval_height, center_offset=center_offset
                        )
                        results[(incubation_time, replicate)] = (key_points, flux_data, cumulative_flux, total_flux, last_tiff, tiff_folder_path, roi_center, roi_mask)
                    except Exception as e:
                        print(f"Error processing {tiff_folder_path}: {str(e)}")
    
    return results


In [51]:
def extract_info_from_folder(folder_name):
    # Extract incubation time and replicate number
    match = re.match(r'(\d+)(?:_min)?_(\d+)_finer', folder_name)
    if match:
        incubation_time = int(match.group(1))
        replicate = int(match.group(2))
        return incubation_time, replicate
    return None, None

def load_piv_data(root_directory):
    data = []
    
    # Iterate through all items in the root directory
    for item in os.listdir(root_directory):
        item_path = os.path.join(root_directory, item)
        if os.path.isdir(item_path) and item.endswith('_finer'):
            # Extract incubation time and replicate number
            incubation_time, replicate = extract_info_from_folder(item)
            
            if incubation_time is not None and replicate is not None:
                velocity_file = os.path.join(item_path, 'velocity_data.pkl')
                
                if os.path.exists(velocity_file):
                    # Load velocity data
                    with open(velocity_file, 'rb') as f:
                        velocity_data = pickle.load(f)
                    
                    # Process velocity data
                    for frame, (ux, uy) in velocity_data.items():
                        mean_velocity = np.sqrt(np.mean(ux**2 + uy**2))
                        max_velocity = np.sqrt(np.max(ux**2 + uy**2))
                        
                        data.append({
                            'folder_name': item,
                            'incubation_time': incubation_time,
                            'replicate': replicate,
                            'frame': frame,
                            'mean_velocity': mean_velocity,
                            'max_velocity': max_velocity,
                            'ux': ux,
                            'uy': uy
                        })
    
    # Create DataFrame
    df = pd.DataFrame(data)
    return df

# Usage
root_directory = "/Users/scliu/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/0104PIV_data_fine"
df = load_piv_data(root_directory)

# Display the first few rows of the DataFrame
print(df.head())

# Get basic statistics
print(df.describe())

# Group by incubation time and replicate, and get mean velocities
summary = df.groupby(['incubation_time', 'replicate']).agg({
    'mean_velocity': 'mean',
    'max_velocity': 'mean'
}).reset_index()

print(summary)

# Usage
root_folder = "/Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/0104"
pixel_size = 0.43  # µm per pixel
num_points = 256
oval_width = 1400  # pixels
oval_height = 500  # pixels
center_offset = (160, 0)  # Adjust these values to move the ROI

# Process all videos
results = process_all_videos(root_folder, df, pixel_size, num_points, oval_width, oval_height, center_offset)


     folder_name  incubation_time  replicate  frame  mean_velocity  \
0  5_min_1_finer                5          1      0       2.138180   
1  5_min_1_finer                5          1      1       2.119352   
2  5_min_1_finer                5          1      2       2.231920   
3  5_min_1_finer                5          1      3       2.279929   
4  5_min_1_finer                5          1      4       2.213320   

   max_velocity                                                 ux  \
0      7.415277  [[2.2125442, 2.9422464, 2.7424815, 1.3800211, ...   
1      7.632476  [[-0.04790093, 0.10697776, -0.42355973, -1.231...   
2     10.642627  [[0.8120931, 0.29884127, -0.8528447, -1.763579...   
3      8.440644  [[0.9665268, -0.38477817, -2.4426982, -3.44039...   
4      8.004435  [[-0.15921363, -0.040638622, 0.265877, 0.38512...   

                                                  uy  
0  [[0.64672816, 0.2536938, -0.5066219, -1.139675...  
1  [[0.096666045, -0.18312608, -0.24286759, -0.0

In [76]:
specific_cases = [
    (10, 3, "/Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/0104PIV_data_0104/10_3_finer/velocity_60.png"),
    (170, 3, "/Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/0104PIV_data_0104/170_3_finer/velocity_60.png")
]

plt.rc('axes', titlesize=24)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('legend', fontsize=22)
plt.rc('figure', titlesize=22)

target_size = (2048, 2048)
roi_shift = (0, 0)  # (x_shift, y_shift) - adjust these values to move ROI left/right and up/down

# First, find the global min and max values for normalization
all_mass_cumulative_flux = []

for (incubation_time, replicate), (key_points, flux_data, cumulative_flux, total_flux, last_tiff, tiff_folder_path, roi_center, roi_mask) in results.items():
    boundary_length = len(key_points)
    mass_cumulative_flux = cumulative_flux * boundary_length
    all_mass_cumulative_flux.extend(mass_cumulative_flux)

global_min_flux = np.min(all_mass_cumulative_flux)
global_max_flux = np.max(all_mass_cumulative_flux)

# Now, plot with the global normalization
for (incubation_time, replicate), (key_points, flux_data, cumulative_flux, total_flux, last_tiff, tiff_folder_path, roi_center, roi_mask) in results.items():
    for case in specific_cases:
        if (incubation_time, replicate) == (case[0], case[1]):
            # Load and resize the specified image
            custom_image_path = case[2]
            with Image.open(custom_image_path) as img:
                original_size = img.size
                print(f"Original image size: {original_size}")
                img_resized = img.resize(target_size, Image.LANCZOS)
                frame = np.array(img_resized)

            # Apply shift to key_points if needed
            key_points_shifted = key_points.copy()
            key_points_shifted[:, 0] += roi_shift[0]
            key_points_shifted[:, 1] += roi_shift[1]

            # Convert density flux to mass flux
            boundary_length = len(key_points)
            mass_cumulative_flux = cumulative_flux * boundary_length

            fig, ax1 = plt.subplots(figsize=(10, 10))
            
            # Oval plot
            ax1.imshow(frame)
            scatter = ax1.scatter(key_points_shifted[:, 0], key_points_shifted[:, 1], c=mass_cumulative_flux, cmap='PiYG_r', s=75, alpha=1, norm=Normalize(vmin=global_min_flux, vmax=global_max_flux))
            cbar = plt.colorbar(scatter, ax=ax1, fraction=0.046, pad=0.04)
            cbar.set_label('Cumulative Mass Flux (MT)', fontsize=18)
            cbar.ax.tick_params(labelsize=16, width=2, length=10)  # Increase the width and length of the tick marks
            cbar.ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
            cbar.ax.yaxis.offsetText.set_fontsize(16)
            cbar.ax.ticklabel_format(style='sci', scilimits=(0,0))

            ax1.axis('off')  # Remove axes and ticks

            plt.tight_layout()
            plt.savefig(f'/Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/flux/flux_plot_{incubation_time}min_rep{replicate}_roi_heatmap_PiYG_r.png', dpi=300, bbox_inches='tight')
            plt.close()

print("ROI imposed plots with heatmap (no title) and color bar with scientific notation and larger tick marks saved.")

def set_plot_style():
    plt.style.use('default')
    plt.rc('font', size=24)
    plt.rc('axes', titlesize=24)
    plt.rc('axes', labelsize=18)
    plt.rc('xtick', labelsize=18)
    plt.rc('ytick', labelsize=18)
    plt.rc('legend', fontsize=22)
    plt.rc('figure', titlesize=22)

    return {
        'FIGURE_WIDTH': 7,
        'FIGURE_HEIGHT': 5,
        'SCATTER_SIZE': 20,
        'SCATTER_ALPHA': 0.7
    }

style_params = set_plot_style()

fig_combined, ax_combined = plt.subplots(figsize=(style_params['FIGURE_WIDTH'], style_params['FIGURE_HEIGHT']))

color_map = plt.cm.viridis
norm = Normalize(vmin=10, vmax=170)  # Adjust these values based on your data range

for (incubation_time, replicate), (key_points, flux_data, cumulative_flux, total_flux, last_tiff, tiff_folder_path, roi_center, roi_mask) in results.items():
    time_points = np.arange(total_flux.shape[0]) * 10 / 60  # Convert frames to minutes
    boundary_length = len(key_points)
    mass_total_flux = total_flux * boundary_length
    color = color_map(norm(incubation_time))
    ax_combined.plot(time_points, mass_total_flux, color=color, alpha=style_params['SCATTER_ALPHA'])

ax_combined.set_xlabel('Time (min)')
ax_combined.set_ylabel('Total Mass Flux (MT/s)')
ax_combined.axhline(0, color='gray', linewidth=0.5, linestyle='--')

ax_combined.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax_combined.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax_combined)
cbar.set_label('Incubation Time (min)')

plt.tight_layout()
plt.savefig('/Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/flux/combined_total_mass_flux_plot_viridis.png', dpi=300, bbox_inches='tight')
plt.close()

print("Combined total mass flux plot saved with viridis color map.")


Original image size: (2310, 2310)
Original image size: (2310, 2310)
ROI imposed plots with heatmap (no title) and color bar with scientific notation and larger tick marks saved.
Combined total mass flux plot saved with viridis color map.


In [74]:
specific_cases = [
    (10, 3, "/Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/0104PIV_data_0104/10_3_finer/velocity_60.png"),
    (170, 3, "/Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/0104PIV_data_0104/170_3_finer/velocity_60.png")
]

plt.rc('axes', titlesize=24)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('legend', fontsize=22)
plt.rc('figure', titlesize=22)

target_size = (2048, 2048)
roi_shift = (0, 0)  # (x_shift, y_shift) - adjust these values to move ROI left/right and up/down

# First, find the global min and max values for normalization
all_mass_cumulative_flux = []

for (incubation_time, replicate), (key_points, flux_data, cumulative_flux, total_flux, last_tiff, tiff_folder_path, roi_center, roi_mask) in results.items():
    boundary_length = len(key_points)
    mass_cumulative_flux = cumulative_flux * boundary_length
    all_mass_cumulative_flux.extend(mass_cumulative_flux)

global_min_flux = np.min(all_mass_cumulative_flux)
global_max_flux = np.max(all_mass_cumulative_flux)

# Now, plot with the global normalization
for (incubation_time, replicate), (key_points, flux_data, cumulative_flux, total_flux, last_tiff, tiff_folder_path, roi_center, roi_mask) in results.items():
    for case in specific_cases:
        if (incubation_time, replicate) == (case[0], case[1]):
            # Load and resize the specified image
            custom_image_path = case[2]
            with Image.open(custom_image_path) as img:
                original_size = img.size
                print(f"Original image size: {original_size}")
                img_resized = img.resize(target_size, Image.LANCZOS)
                frame = np.array(img_resized)

            # Apply shift to key_points if needed
            key_points_shifted = key_points.copy()
            key_points_shifted[:, 0] += roi_shift[0]
            key_points_shifted[:, 1] += roi_shift[1]

            # Convert density flux to mass flux
            boundary_length = len(key_points)
            mass_cumulative_flux = cumulative_flux * boundary_length

            fig, ax1 = plt.subplots(figsize=(10, 10))
            
            # Oval plot
            ax1.imshow(frame)
            scatter = ax1.scatter(key_points_shifted[:, 0], key_points_shifted[:, 1], c=mass_cumulative_flux, cmap='coolwarm', s=75, alpha=1, norm=Normalize(vmin=global_min_flux, vmax=global_max_flux))

            cbar = plt.colorbar(scatter, ax=ax1, fraction=0.046, pad=0.04)
            cbar.set_label('Cumulative Mass Flux (MT)', fontsize=32)
            cbar.ax.tick_params(labelsize=28, width=2, length=10)  # Increase the width and length of the tick marks
            cbar.ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
            cbar.ax.yaxis.offsetText.set_fontsize(16)
            cbar.ax.ticklabel_format(style='sci', scilimits=(0,0))

            ax1.axis('off')  # Remove axes and ticks

            plt.tight_layout()
            plt.savefig(f'/Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_1/flux/flux_plot_{incubation_time}min_rep{replicate}_roi_heatmap_PiYG_r.png', dpi=300, bbox_inches='tight')
            plt.close()

print("ROI imposed plots with heatmap (no title) and color bar with scientific notation and larger tick marks saved.")


Original image size: (2310, 2310)
Original image size: (2310, 2310)
ROI imposed plots with heatmap (no title) and color bar with scientific notation and larger tick marks saved.
